# Working with OTU tables

There are many ways to manipulate OTU tables in QIIME, including filtering, sorting, splitting, rarefying, merging, and summarizing. In this tutorial, we'll see how to filter OTUs based on their abundance, filter samples based on their metadata, sort samples based on taxonomic abundance, split an OTU table based on taxonomy, and filter an OTU table based on taxonomy.

## Getting started

Let's create a new directory where we'll try out a few exercises. For convenience, we'll copy the open-reference OTU table and mapping file we created in the QIIME Illumina Overview Tutorial. Note that we're renaming the files to have shorter names (also for convenience and clarity in the following exercises).

In [ ]:
from os import chdir
from os.path import join
from functools import partial
from IPython.display import FileLinks, FileLink

chdir(working_dir)

!mkdir -p otu-table-exercises
!cp moving_pictures_tutorial-1.9.0/illumina/otus/otu_table_mc2_w_tax_no_pynast_failures.biom otu-table-exercises/otu_table.biom
!cp moving_pictures_tutorial-1.9.0/illumina/map.tsv otu-table-exercises/map.tsv

tutorial_dir = 'otu-table-exercises/'
chdir(tutorial_dir)
FileLink = partial(FileLink, url_prefix=join('exercises', name, tutorial_dir))
FileLinks = partial(FileLinks, url_prefix=join('exercises', name, tutorial_dir))

## Filtering OTUs

First, let's practice filtering OTUs from an OTU table. Let's filter out all *singleton* and *doubleton* OTUs from the table, keeping only OTUs that have a count of 3 or more across all samples:

In [ ]:
!filter_otus_from_otu_table.py -o otu_table_min_count_3.biom -i otu_table.biom -n 3

Let's generate summaries of our original and filtered OTU tables:

In [ ]:
!biom summarize-table -i otu_table.biom

In [ ]:
!biom summarize-table -i otu_table_min_count_3.biom

Note that there are fewer OTUs in the filtered table than the original (look for the *Num observations* field at the top of the summary).

## Filtering samples

In the previous exercise, we filtered OTUs from our OTU table. We can also filter samples from an OTU table based on their metadata. Recall that the taxa summary plots in the QIIME Illumina Overview Tutorial had the samples grouped by four sample types (left/right palm, gut, and tongue). Let's filter out samples so that our new table only contains gut and tongue samples:

In [ ]:
!filter_samples_from_otu_table.py -o otu_table_gut_tongue.biom -i otu_table.biom -m map.tsv -s 'SampleType:gut,tongue'

For more details about how to specify what samples to retain/discard based on their metadata, see QIIME's [sample metadata description tutorial](http://qiime.org/tutorials/metadata_description.html).

Next, let's create taxa summary plots using our filtered OTU table, grouping samples by their type:

In [ ]:
!summarize_taxa_through_plots.py -o taxa_plots_gut_tongue -i otu_table_gut_tongue.biom -m map.tsv -c SampleType

Open up the resulting bar charts. Note that there are only bars for gut and tongue samples; left/right palm samples have been excluded.

In [ ]:
FileLink('taxa_plots_gut_tongue/taxa_summary_plots/bar_charts.html')

## Sorting samples

Next, let's sort the samples in our original OTU table based on relative abundance of the Bacteroidetes phylum. To do this, we first need to add the relative abundances of each phyla to our mapping file. We pass `-L 2` to specify taxonomic level 2, which is phylum level in the case of Greengenes taxonomies:

In [ ]:
!summarize_taxa.py -o taxa_summaries_L2_mapping_file -i otu_table.biom -m map.tsv -L 2

The new mapping file with phyla relative abundances is created in the `taxa_summaries_L2_mapping_file` directory.

Now that we have columns of relative abundances in our mapping file, we can sort the samples in the OTU table by Bacteroidetes relative abundances:

In [ ]:
!sort_otu_table.py -o otu_table_sorted_bacteroidetes.biom -i otu_table.biom -m taxa_summaries_L2_mapping_file/map_L2.txt -s 'k__Bacteria;p__Bacteroidetes'

Finally, let's create taxa summary plots using our sorted table. We don't provide a mapping file or column because we don't want to group samples by metadata:

In [ ]:
!summarize_taxa_through_plots.py -o taxa_plots_sorted_bacteroidetes -i otu_table_sorted_bacteroidetes.biom

Open up the resulting area charts. The samples should be sorted by lowest to highest abundance of Bacteroidetes.

In [ ]:
FileLink('taxa_plots_sorted_bacteroidetes/taxa_summary_plots/area_charts.html')

## Splitting by taxonomy

Let's split our original OTU table at the phylum level, creating a new OTU table for each phylum in the original table. Just like before, we specify `-L 2` to split at the phylum level:

In [ ]:
!split_otu_table_by_taxonomy.py -o split_otu_table_L2 -i otu_table.biom -L 2

To create taxa summary plots using the OTU table containing only Bacteroidetes, we run:

In [ ]:
!summarize_taxa_through_plots.py -o taxa_plots_bacteroidetes -i split_otu_table_L2/otu_table_k__Bacteria_p__Bacteroidetes.biom -m map.tsv -c SampleType

In [ ]:
FileLink('taxa_plots_bacteroidetes/taxa_summary_plots/bar_charts.html')

## Filtering taxa

As a final example, let's filter out all Firmicutes from our original OTU table:

In [ ]:
!filter_taxa_from_otu_table.py -o otu_table_no_firmicutes.biom -i otu_table.biom -n 'p__Firmicutes'

To create taxa summary plots, we run:

In [ ]:
!summarize_taxa_through_plots.py -o taxa_plots_no_firmicutes -i otu_table_no_firmicutes.biom -m map.tsv -c SampleType

Open up the resulting bar charts. Note that Firmicutes aren't present in the taxa summary plots because they were filtered out of the OTU table.

In [ ]:
FileLink('taxa_plots_no_firmicutes/taxa_summary_plots/bar_charts.html')

There are many other ways to manipulate OTU tables in QIIME. For more details, see QIIME's [working with BIOM tables tutorial](http://qiime.org/tutorials/working_with_biom_tables.html).

## Exercises

1. Filter the original OTU table (`otu_table.biom`) to include only OTUs that have a count less than 3 across all samples. How many OTUs are left in the table?

2. Filter the original OTU table (`otu_table.biom`) to remove samples that were taken during antibiotic usage. How many samples were removed?

   **Hint:** The relevant column denoting antibiotic usage is `ReportedAntibioticUsage`. This column contains the values `Yes` and `No`.